# Personal Budget Data Preparation

## Plan

### income_log table

1. Import raw income_log data (two)
2. Combine dataframes to one dataframe
3. Sort resulting dataframe by date
4. Graph data to validate distribution
5. Export data to sql file with prefix `final_`

### savings_log table

1. Import raw savings_log data (two)
2. Sort each dataframe by date
3. Calculate sum of `change` column per dataframe
4. If sum is negative, change some cells in the `change` column to positive
5. Add new cumulative column called `balance` for each dataframe using `cumsum`
6. Combine dataframes to one dataframe
7. Try: Move `balance` column to be after `change` column
8. Sort resulting dataframe by date
9. Export dataframe to sql file with prefix `final_`

### users table

Write sql file containing two users each with `name`, `username`, and `password`

## Implementation

In [ ]:
import numpy as np
import pandas as pd
import 